# Projet ML
GAUD - CURRAL - GALMIER

## Libraries

### Installation

In [1]:
! pip install tensorflow
! pip install resampy
! pip install librosa

### Import

In [2]:
import numpy as np
import tensorflow as tf
from glob import glob
from tensorflow import audio

import numpy as np
# import required modules 
import os
import matplotlib.pyplot as plt

import librosa
import librosa.display
import librosa.feature

import numpy as np
import pandas as pd
import resampy

### Variables

In [3]:
TEST_DATA_PATH          = 'Data\\test'
TRAIN_DATA_PATH         = 'Data\\train'
VAL_DATA_PATH           = 'Data\\val'

MIN_NUM_FILES           = 160

# Code

In [4]:
import os
#The goal is to sort the bird datasets by number of files to cut the birds with too few files.

birds = []
trainFileNames = []

folders = os.listdir(TRAIN_DATA_PATH)
sorted_folders = sorted(folders, key=lambda folder: len(os.listdir(os.path.join(TRAIN_DATA_PATH, folder))), reverse=True)

for folder in sorted_folders:
    folder_path = os.path.join(TRAIN_DATA_PATH, folder)
    num_files = len(os.listdir(folder_path))
    if num_files > MIN_NUM_FILES:
        birds.append(folder)
        for file in os.listdir(folder_path):
            trainFileNames.append(os.path.join(folder_path, file))



### Limiting the folders in test

In [5]:
folders = os.listdir(TEST_DATA_PATH)

testFileNames = []

for folder in folders:
    if folder in birds:
        folder_path = os.path.join(TEST_DATA_PATH, folder)
        for file in os.listdir(folder_path):
            testFileNames.append(os.path.join(folder_path, file))

### Limiting the folders in Val

In [6]:
folders = os.listdir(VAL_DATA_PATH)

valFileNames = []

for folder in folders:
    if folder in birds:
        folder_path = os.path.join(VAL_DATA_PATH, folder)
        for file in os.listdir(folder_path):
            valFileNames.append(os.path.join(folder_path, file))

In [7]:
trainFileNames      =   tf.random.shuffle(trainFileNames)
testFileNames       =   tf.random.shuffle(testFileNames)
valFileNames        =   tf.random.shuffle(valFileNames )

In [29]:
def mapFileNamesToLabels(fileNames):
    fileNames_list = fileNames.numpy().tolist()
    return list(map(lambda x: x.decode('utf-8'), trainFileNames_list))

In [30]:
trainFileNamesMapped = mapFileNamesToLabels(trainFileNames)


In [8]:
firstTrainFile = tf.io.read_file(trainFileNames[0])

In [9]:
test_audio, samplerate = tf.audio.decode_wav(contents=firstTrainFile)

In [10]:
def decode_audio(audio_binary):
    audio, _ = tf.audio.decode_wav(contents=audio_binary)
    return audio

import os

def get_label_from_filepath(file_path):
    parts = tf.strings.split(input=file_path, sep=os.path.sep)
    return parts[-2]

def get_waveform_and_label(file_path):
    label = get_label_from_filepath(file_path)
    audio_binary = tf.io.read_file(file_path)
    waveform = decode_audio(audio_binary)
    waveform = tf.reduce_mean(waveform, axis=1)
    #waveform = tf.squeeze(waveform, axis=-1)
    return waveform, label

code is a function named `decode_audio` that takes in binary audio data and decodes it into a tensor using TensorFlow's audio decoding function.

The function `decode_audio` accepts one parameter, `audio_binary`, which is expected to be the binary data of a WAV audio file.

Inside the function, the `tf.audio.decode_wav` function is called with `audio_binary` as its argument. This function decodes the WAV audio file into a 2D tensor where the first dimension represents the time axis and the second dimension represents the channels. It returns two items: a tensor that contains the decoded audio data and the sample rate of the audio file. In this case, the sample rate is not used, hence it is assigned to `_`, a throwaway variable.


In [11]:
#t = get_label_from_filepath(trainFileNames)

In [12]:
AUTOTUNE = tf.data.AUTOTUNE

In [20]:
trainFileNames

<tf.Tensor: shape=(4359,), dtype=string, numpy=
array([b'Data\\train\\Common Wood Pigeon\\XC629531 5.wav',
       b'Data\\train\\Asian Koel\\XC301717 1.wav',
       b'Data\\train\\Red-billed Blue Magpie\\XC299810 0.wav', ...,
       b'Data\\train\\Black Kite\\XC477874 25.wav',
       b'Data\\train\\Red-billed Blue Magpie\\XC396236 2.wav',
       b'Data\\train\\Common Wood Pigeon\\XC725376 0.wav'], dtype=object)>

The `num_parallel_calls` parameter is set to `AUTOTUNE`, which is a special value that prompts TensorFlow to choose the optimal number of parallel calls dynamically at runtime, based on available CPU resources. This can help improve the speed of loading data.

In [32]:
trainFilesDS        = tf.data.Dataset.from_tensor_slices(trainFileNamesMapped)
testFilesDS         = tf.data.Dataset.from_tensor_slices(testFileNames)
valFilesDS          = tf.data.Dataset.from_tensor_slices(valFileNames)

In [34]:
print(trainFilesDS)

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [21]:
def extract_features(file_name):
    print("Extraction des caractéristiques de: ", file_name)
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=20)
        mfccs_processed = np.mean(mfccs.T,axis=0)

    except Exception as e:
        print("Erreur rencontrée lors de l'extraction des caractéristiques: ", file_name)
        return None

    return mfccs_processed

In [35]:
trainFeatureDS      = trainFilesDS.map(map_func=extract_features, num_parallel_calls=AUTOTUNE)
testFeatureDS       = testFileNames.map(map_func=extract_features, num_parallel_calls=AUTOTUNE)
valFeatureDS        = valFilesDS.map(map_func=extract_features, num_parallel_calls=AUTOTUNE)

Extraction des caractéristiques de:  Tensor("args_0:0", shape=(), dtype=string)
Erreur rencontrée lors de l'extraction des caractéristiques:  Tensor("args_0:0", shape=(), dtype=string)


InvalidArgumentError: Length for attr 'output_types' of 0 must be at least minimum 1
	; NodeDef: {{node ParallelMapDatasetV2}}; Op<name=ParallelMapDatasetV2; signature=input_dataset:variant, other_arguments:, num_parallel_calls:int64 -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=deterministic:string,default="default"; attr=preserve_cardinality:bool,default=false; attr=metadata:string,default=""> [Op:ParallelMapDatasetV2] name: 

In [ ]:
train_ds    = trainFeatureDS
val_ds      = testFeatureDS
test_ds     = valFeatureDS

In [ ]:
batch_size      = 64
train_ds        = train_ds.batch(batch_size)
val_ds          = val_ds.batch(batch_size)

In [ ]:
train_ds        = train_ds.cache().prefetch(AUTOTUNE)
val_ds          = val_ds.cache().prefetch(AUTOTUNE)

In [ ]:
for spectrogram, _ in spectrogram_ds.take(1):
    input_shape = spectrogram.shape

num_labels = len(birds)